In [2]:
#!/usr/bin/env python
import os
import torch
import numpy as np
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

# Pretpostavimo da je ista arhitektura definirana
from brevitas.nn import QuantConv2d, QuantLinear, QuantReLU, QuantMaxPool2d

# Definicija iste arhitekture (morate imati isto definirane težine)
class QuantVGG8(torch.nn.Module):
    def __init__(self, in_channels=3, bit_width=4, num_classes=10, img_size=32):
        super(QuantVGG8, self).__init__()
        self.features = torch.nn.Sequential(
            QuantConv2d(in_channels, 16, kernel_size=3, padding=1, weight_bit_width=bit_width),
            QuantReLU(bit_width=bit_width),
            QuantMaxPool2d(kernel_size=2),
            QuantConv2d(16, 32, kernel_size=3, padding=1, weight_bit_width=bit_width),
            QuantReLU(bit_width=bit_width),
            QuantMaxPool2d(kernel_size=2),
            QuantConv2d(32, 64, kernel_size=3, padding=1, weight_bit_width=bit_width),
            QuantReLU(bit_width=bit_width),
            QuantMaxPool2d(kernel_size=2),
        )
        feature_map_size = img_size // 8
        self.classifier = torch.nn.Sequential(
            QuantLinear(64 * (feature_map_size**2), 128, bias=True, weight_bit_width=bit_width),
            QuantReLU(bit_width=bit_width),
            QuantLinear(128, num_classes, bias=True, weight_bit_width=bit_width)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

def asymmetric_quantize(arr, num_bits=8):
    qmin = 0
    qmax = 2**num_bits - 1
    beta = np.min(arr)
    alpha = np.max(arr)
    scale = (alpha - beta) / qmax if (alpha-beta)!=0 else 1.0
    zero_point = np.clip(-beta/scale, 0, qmax).round().astype(np.int8)
    quantized_arr = np.clip(np.round(arr / scale + zero_point), qmin, qmax).astype(np.float32)
    return quantized_arr

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    # Inicijalizacija modela
    model = QuantVGG8(in_channels=3, bit_width=4, num_classes=10, img_size=32)
    model.load_state_dict(torch.load("model_vgg8.pth", map_location=torch.device('cpu')))
    model = model.to(device)
    model.eval()
    print("Model loaded from 'model_vgg8.pth'.")

    # Postavke za export
    root_dir = "./export"
    os.makedirs(root_dir, exist_ok=True)
    onnx_filename = os.path.join(root_dir, "part1.onnx")
    onnx_clean_filename = os.path.join(root_dir, "part1_clean.onnx")
    finn_filename = os.path.join(root_dir, "ready_finn.onnx")

    # Kreiramo dummy ulaz – primjer za CIFAR-10 (3 kanala, 32x32)
    dummy_np = np.random.rand(1, 3, 32, 32).astype(np.float32)
    dummy_np = asymmetric_quantize(dummy_np, num_bits=8)
    print("Max value in quantized input:", np.max(dummy_np))
    scale = 1.0
    input_t = torch.from_numpy(dummy_np * scale)

    print("Exporting model to QONNX format...")
    try:
        export_qonnx(model, export_path=onnx_filename, input_t=input_t)
        print(f"Model exported to ONNX file: {onnx_filename}")
    except Exception as e:
        print("Error during ONNX export:", e)

    print("Cleaning ONNX model with qonnx_cleanup...")
    try:
        qonnx_cleanup(onnx_filename, out_file=onnx_clean_filename)
        print(f"Cleaned ONNX model saved to: {onnx_clean_filename}")
    except Exception as e:
        print("Error during ONNX model cleanup:", e)

    print("Converting QONNX to FINN model...")
    try:
        model_wrapper = ModelWrapper(onnx_clean_filename)
        model_wrapper = model_wrapper.transform(ConvertQONNXtoFINN())
        model_wrapper.save(finn_filename)
        print("FINN model saved to:", finn_filename)
    except Exception as e:
        print("Error during FINN conversion:", e)

if __name__ == '__main__':
    main()


Using device: cpu
Model loaded from 'model_vgg8.pth'.
Max value in quantized input: 255.0
Exporting model to QONNX format...


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


Model exported to ONNX file: ./export/part1.onnx
Cleaning ONNX model with qonnx_cleanup...
Cleaned ONNX model saved to: ./export/part1_clean.onnx
Converting QONNX to FINN model...
FINN model saved to: ./export/ready_finn.onnx


/home/davorin/Documents/finn/deps/qonnx/src/qonnx/transformation/gemm_to_matmul.py:57: UserWarning: The GemmToMatMul transformation only offers explicit support for version 9 of the Gemm node, but the ONNX version of the supplied model is 14. Thus the transformation may fail or return incomplete results.
  warnings.warn(
